Distance and Boudaries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_face_detection = mp.solutions.face_detection

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands, mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Hand detections
        hand_results = hands.process(image)
        
        # Face detections
        face_results = face_detection.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Hand detections
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                           mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                           mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
                
                landmarks = np.array([[lm.x * width, lm.y * height] for lm in hand_landmarks.landmark])
                hand_bbox = cv2.boundingRect(landmarks.astype(np.float32))
                
                # Draw bounding box around hand
                print("Bounding Box",hand_bbox)
                cv2.rectangle(image, (int(hand_bbox[0]), int(hand_bbox[1])), 
                              (int(hand_bbox[0]+hand_bbox[2]), int(hand_bbox[1]+hand_bbox[3])), 
                              (0, 255, 0), 2)
                
                # Extract tip of the hand (assuming it's at the bottom when held vertically)
                tip_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
                
                # Convert tip landmark to image coordinates
                height, width, _ = image.shape
                tip_x = int(tip_landmark.x * width)
                tip_y = int(tip_landmark.y * height)
                
                # Hand position
                hand_position = (tip_x, tip_y)
                
                # Face detections
                if face_results.detections:
                    for detection in face_results.detections:
                        bboxC = detection.location_data.relative_bounding_box
                        
                        # Convert normalized bounding box coordinates to pixels
                        ih, iw, _ = image.shape
                        bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                               int(bboxC.width * iw), int(bboxC.height * ih)
                        
                        # Draw bounding box around face
                        cv2.rectangle(image, bbox, (255, 0, 0), 2)
                        
                        # Face position (considering the center of the bounding box)
                        face_position = (bbox[0] + bbox[2] // 2, bbox[1] + bbox[3] // 2)
                        
                        # Calculate distance between hand tip and face position
                        distance = math.sqrt((hand_position[0] - face_position[0])**2 + (hand_position[1] - face_position[1])**2)
                        
                        # Display distance on the image
                        cv2.putText(image, f"Distance: {distance:.2f} pixels", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Bounding Box (401, 230, 233, 266)
Bounding Box (339, 190, 237, 294)
Bounding Box (313, 167, 262, 306)
Bounding Box (285, 156, 250, 312)
Bounding Box (278, 152, 245, 317)
Bounding Box (276, 151, 247, 318)
Bounding Box (273, 151, 244, 319)
Bounding Box (271, 151, 247, 318)
Bounding Box (269, 151, 243, 319)
Bounding Box (268, 151, 244, 318)
Bounding Box (269, 148, 239, 320)
Bounding Box (273, 147, 236, 317)
Bounding Box (272, 148, 238, 316)
Bounding Box (281, 148, 232, 314)
Bounding Box (279, 149, 233, 314)
Bounding Box (296, 160, 220, 301)
Bounding Box (299, 159, 217, 303)
Bounding Box (352, 201, 168, 262)
Bounding Box (373, 218, 147, 238)
Bounding Box (373, 218, 148, 239)
Bounding Box (379, 227, 138, 221)
Bounding Box (383, 223, 137, 227)
Bounding Box (383, 223, 136, 224)
Bounding Box (383, 224, 136, 222)
Bounding Box (386, 227, 133, 216)
Bounding Box (383, 226, 135, 216)
Bounding Box (384, 226, 134, 216)
Bounding Box (384, 226, 133, 214)
Bounding Box (385, 226, 132, 214)
Bounding Box (